<img src="header.jpg" alt="Wide World Importers" width="400"/>
<img src="iStock-1328873668.jpg" alt="Wide World Importers - Solar Panel" width="500"/>

## Setup


In [1]:
import json
import logging
from datetime import datetime

import azureml.core
import numpy as np
import pandas as pd
from azureml.automl.core.featurization import FeaturizationConfig
from azureml.core import Dataset, Experiment, Workspace
from azureml.train.automl import AutoMLConfig

This notebook is compatible with Azure ML SDK version 1.35.0 or later.

In [2]:
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

You are currently using version 1.48.0 of the Azure ML SDK


As part of the setup you have already created a <b>Workspace</b>. To run AutoML, you also need to create an <b>Experiment</b>. An Experiment corresponds to a prediction problem you are trying to solve, while a Run corresponds to a specific approach to the problem.

In [3]:
ws = Workspace.from_config()

# choose a name for the run history container in the workspace
experiment_name = "demand_forecast_exp"

experiment = Experiment(ws, experiment_name)

output = {}
output["Subscription ID"] = ws.subscription_id
output["Workspace"] = ws.name
output["SKU"] = ws.sku
output["Resource Group"] = ws.resource_group
output["Location"] = ws.location
output["Run History Name"] = experiment_name
output["SDK Version"] = azureml.core.VERSION
pd.set_option("display.max_colwidth", None)
outputDf = pd.DataFrame(data=output, index=[""])
outputDf.T

,
Subscription ID,506e86fc-853c-4557-a6e5-ad72114efd2b
Workspace,amlws-midp
SKU,Basic
Resource Group,rg-midpwithazurecosmos-prod
Location,eastus2
Run History Name,demand_forecast_exp
SDK Version,1.48.0


## Compute
You will need to create a [compute target](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute) for your AutoML run. In this tutorial, you create AmlCompute as your training compute resource.

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

#### Creation of AmlCompute takes approximately 5 minutes. 
If the resource with AmlCompute name already exists in your workspace, this code will skip the creation process.
As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your cluster.
amlcompute_cluster_name = "wwi-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_DS12_V2", max_nodes=4
    )
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Data

The [Machine Learning service workspace](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-workspace) is paired with the storage account, which contains the default data store. We will use it to upload the solar panel demand data and create [tabular dataset](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.tabulardataset?view=azure-ml-py) for training. A tabular dataset defines a series of lazily-evaluated, immutable operations to load data from the data source into tabular representation.

In [5]:
datastore = ws.get_default_datastore()
datastore.upload_files(
    files=["solar-panel-demand-no.csv"], target_path="dataset/", overwrite=True, show_progress=True
)

"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


$AZUREML_DATAREFERENCE_19a0bdf8638b4dd7ab381fbb993f7840

Let's set up what we know about the dataset. 

**Target column** is what we want to forecast.

**Time column** is the time axis along which to predict.

In [6]:
target_column_name = "cnt"
time_column_name = "date"

In [7]:
dataset = Dataset.Tabular.from_delimited_files(
    path=[(datastore, "dataset/solar-panel-demand-no.csv")]
).with_timestamp_columns(fine_grain_timestamp=time_column_name)

# Drop the columns 'casual' and 'registered' as these columns are a breakdown of the total and therefore a leak.
dataset = dataset.drop_columns(columns=["casual", "registered"])

dataset.take_sample(0.01).to_pandas_dataframe().reset_index(drop=True)

,instant,date,season,yr,mnth,weekday,weathersit,incentive_level,incentive_duration_remaining,power_grid_premium,power_grid_risk,cnt
0,23,2011-01-23,1,0,1,0,1,0.096522,0.098839,0.436522,0.246600,986
1,28,2011-01-28,1,0,1,5,2,0.203478,0.223317,0.793043,0.123300,1167
2,208,2011-07-27,3,0,7,3,1,0.775000,0.690667,0.402917,0.183463,4656
3,230,2011-08-18,3,0,8,4,1,0.711667,0.662258,0.654583,0.233208,3805
4,440,2012-03-15,1,1,3,4,1,0.557500,0.532825,0.579583,0.149883,6192
5,693,2012-11-23,4,1,11,5,1,0.368333,0.378779,0.568750,0.148021,3910


### Split the data

We first split the dataset into train and test sets. Note that we are splitting on the basis of time. Data before 9/1 will be used for training, and data after and including 9/1 will be used for testing.

In [8]:
# select data that occurs before a specified date
train = dataset.time_before(datetime(2012, 8, 31), include_boundary=True)
train.to_pandas_dataframe().sample(5).reset_index(drop=True)

,instant,date,season,yr,mnth,weekday,weathersit,incentive_level,incentive_duration_remaining,power_grid_premium,power_grid_risk,cnt
0,321,2011-11-17,4,0,11,4,2,0.341667,0.323221,0.575833,0.305362,3053
1,331,2011-11-27,4,0,11,0,1,0.459167,0.455800,0.698333,0.208954,3071
2,354,2011-12-20,4,0,12,2,2,0.385833,0.396454,0.595417,0.061571,3750
3,81,2011-03-22,2,0,3,2,1,0.441667,0.440642,0.624583,0.225750,2703
4,37,2011-02-06,1,0,2,0,1,0.285833,0.291671,0.568333,0.141800,1623


In [9]:
test = dataset.time_after(datetime(2012, 9, 1), include_boundary=True)
test.to_pandas_dataframe().sample(5).reset_index(drop=True)

,instant,date,season,yr,mnth,weekday,weathersit,incentive_level,incentive_duration_remaining,power_grid_premium,power_grid_risk,cnt
0,624,2012-09-15,3,1,9,6,1,0.608333,0.585867,0.501667,0.247521,8714
1,643,2012-10-04,4,1,10,4,2,0.657500,0.607975,0.722917,0.117546,7328
2,652,2012-10-13,4,1,10,6,1,0.393333,0.391396,0.494583,0.146142,7109
3,630,2012-09-21,3,1,9,5,1,0.599167,0.571971,0.668750,0.154229,8167
4,664,2012-10-25,4,1,10,4,2,0.550000,0.529688,0.800417,0.124375,7359


## Forecasting Parameters
To define forecasting parameters for your experiment training, you can leverage the ForecastingParameters class. The table below describes the forecasting parameter we will pass into our experiment.

|Property|Description|
|-|-|
|**time_column_name**|The name of your time column.|
|**forecast_horizon**|The forecast horizon is how many periods forward you would like to forecast. This integer horizon is in units of the time series frequency (e.g. daily, weekly).|
|**country_or_region_for_holidays**|The country/region used to generate holiday features. These should be ISO 3166 two-letter country/region codes (i.e. 'US', 'GB').|
|**target_lags**|The target_lags specifies how far back we will construct the lags of the target variable.|
|**freq**|Forecast frequency. This optional parameter represents the period with which the forecast is desired, for example, daily, weekly, yearly, etc. Use this parameter for the correction of time series containing irregular data points or for padding of short time series. The frequency needs to be a pandas offset alias. Please refer to [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects) for more information.
|**cv_step_size**|Number of periods between two consecutive cross-validation folds. The default value is "auto", in which case AutoML determines the cross-validation step size automatically, if a validation set is not provided. Or users could specify an integer value.

## Train

Instantiate a AutoMLConfig object. This defines the settings and data used to run the experiment.

|Property|Description|
|-|-|
|**task**|forecasting|
|**primary_metric**|This is the metric that you want to optimize.<br> Forecasting supports the following primary metrics <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i>
|**blocked_models**|Models in blocked_models won't be used by AutoML. All supported models can be found at [here](https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.constants.supportedmodels.forecasting?view=azure-ml-py).|
|**experiment_timeout_hours**|Experimentation timeout in hours.|
|**training_data**|Input dataset, containing both features and label column.|
|**label_column_name**|The name of the label column.|
|**compute_target**|The remote compute for training.|
|**n_cross_validations**|Number of cross-validation folds to use for model/pipeline selection. The default value is "auto", in which case AutoMl determines the number of cross-validations automatically, if a validation set is not provided. Or users could specify an integer value.
|**enable_early_stopping**|If early stopping is on, training will stop when the primary metric is no longer improving.|
|**forecasting_parameters**|A class that holds all the forecasting related parameters.|

This notebook uses the blocked_models parameter to exclude some models that take a longer time to train on this dataset. You can choose to remove models from the blocked_models list but you may need to increase the experiment_timeout_hours parameter value to get results.

### Setting forecaster maximum horizon 

The forecast horizon is the number of periods into the future that the model should predict. Here, we set the horizon to 14 periods (i.e. 14 days). Notice that this is much shorter than the number of days in the test set; we will need to use a rolling test to evaluate the performance on the whole test set. 

In [10]:
forecast_horizon = 14

### Convert prediction type to integer
The featurization configuration can be used to change the default prediction type from decimal numbers to integer. This customization can be used in the scenario when the target column is expected to contain whole values as the number of Solar Panels sold per day.

In [11]:
featurization_config = FeaturizationConfig()
# Force the target column, to be integer type.
featurization_config.add_prediction_transform_type("Integer")

### Config AutoML

In [12]:
from azureml.automl.core.forecasting_parameters import ForecastingParameters

forecasting_parameters = ForecastingParameters(
    time_column_name=time_column_name,
    freq="D",  # Set the forecast frequency to be daily
    cv_step_size="auto",
)

automl_config = AutoMLConfig(
    task="forecasting",
    primary_metric="normalized_root_mean_squared_error",
    featurization=featurization_config,
    blocked_models=["ExtremeRandomTrees"],
    experiment_timeout_hours=0.3,
    training_data=train,
    label_column_name=target_column_name,
    compute_target=compute_target,
    enable_early_stopping=True,
    n_cross_validations="auto",  # Feel free to set to a small integer (>=2) if runtime is an issue.
    max_concurrent_iterations=4,
    max_cores_per_iteration=-1,
    verbosity=logging.INFO,
    forecasting_parameters=forecasting_parameters,
)

We will now run the experiment, you can go to Azure ML portal to view the run details. 

In [13]:
remote_run = experiment.submit(automl_config, show_output=False)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
demand_forecast_exp,AutoML_4ff0c6b7-02d8-400c-936f-625078ced73c,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [14]:
remote_run.wait_for_completion()

### Retrieve the Best Run details
Below we retrieve the best Run object from among all the runs in the experiment.

In [ ]:
best_run = remote_run.get_best_child()
best_run

## Featurization

We can look at the engineered feature names generated in time-series featurization via the JSON file named 'engineered_feature_names.json' under the run outputs. Note that a number of named holiday periods are represented. We recommend that you have at least one year of data when using this feature to ensure that all yearly holidays are captured in the training featurization.

In [ ]:
# Download the JSON file locally
best_run.download_file(
    "outputs/engineered_feature_names.json", "engineered_feature_names.json"
)
with open("engineered_feature_names.json", "r") as f:
    records = json.load(f)

records

### View the featurization summary

You can also see what featurization steps were performed on different raw features in the user data. For each raw feature in the user data, the following information is displayed:

- Raw feature name
- Number of engineered features formed out of this raw feature
- Type detected
- If feature was dropped
- List of feature transformations for the raw feature

In [ ]:
# Download the featurization summary JSON file locally
best_run.download_file(
    "outputs/featurization_summary.json", "featurization_summary.json"
)

# Render the JSON as a pandas DataFrame
with open("featurization_summary.json", "r") as f:
    records = json.load(f)
fs = pd.DataFrame.from_records(records)

# View a summary of the featurization
fs[
    [
        "RawFeatureName",
        "TypeDetected",
        "Dropped",
        "EngineeredFeatureCount",
        "Transformations",
    ]
]

## Evaluate

We now use the best fitted model from the AutoML Run to make forecasts for the test set. We will do batch scoring on the test dataset which should have the same schema as training dataset.

The scoring will run on a remote compute. In this example, it will reuse the training compute.

In [ ]:
test_experiment = Experiment(ws, experiment_name + "_test")

### Retrieving forecasts from the model
To run the forecast on the remote compute we will use a helper script: forecasting_script. This script contains the utility methods which will be used by the remote estimator. We copy the script to the project folder to upload it to remote compute.

In [ ]:
import os
import shutil

script_folder = os.path.join(os.getcwd(), "forecast")
os.makedirs(script_folder, exist_ok=True)
shutil.copy("forecasting_script.py", script_folder)

For brevity, we have created a function called run_forecast that submits the test data to the best model determined during the training run and retrieves forecasts. The test set is longer than the forecast horizon specified at train time, so the forecasting script uses a so-called rolling evaluation to generate predictions over the whole test set. A rolling evaluation iterates the forecaster over the test set, using the actuals in the test set to make lag features as needed. 

In [ ]:
from run_forecast import run_rolling_forecast

remote_run = run_rolling_forecast(
    test_experiment, compute_target, best_run, test, target_column_name
)
remote_run

In [ ]:
remote_run.wait_for_completion(show_output=False)

### Download the prediction result for metrics calculation
The test data with predictions are saved in artifact outputs/predictions.csv. You can download it and calculate some error metrics for the forecasts and vizualize the predictions vs. the actuals.

In [ ]:
remote_run.download_file("outputs/predictions.csv", "solar-panel-demand-predictions.csv")
fcst_df = pd.read_csv("solar-panel-demand-predictions.csv")

Note that the rolling forecast can contain multiple predictions for each date, each from a different forecast origin. For example, consider 2012-09-05:

In [ ]:
fcst_df[fcst_df.date == "2012-09-05"]

Here, the forecast origin refers to the latest date of actuals available for a given forecast. The earliest origin in the rolling forecast, 2012-08-31, is the last day in the training data. For origin date 2012-09-01, the forecasts use actual recorded counts from the training data *and* the actual count recorded on 2012-09-01. Note that the model is not retrained for origin dates later than 2012-08-31, but the values for model features, such as lagged values of daily count are updated.

Let's calculate the metrics over all rolling forecasts:

In [ ]:
from azureml.automl.core.shared import constants
from azureml.automl.runtime.shared.score import scoring
from sklearn.metrics import mean_absolute_error, mean_squared_error

# use automl metrics module
scores = scoring.score_regression(
    y_test=fcst_df[target_column_name],
    y_pred=fcst_df["predicted"],
    metrics=list(constants.Metric.SCALAR_REGRESSION_SET),
)

print("[Test data scores]\n")
for key, value in scores.items():
    print("{}:   {:.3f}".format(key, value))

For more details on what metrics are included and how they are calculated, please refer to [supported metrics](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-understand-automated-ml#regressionforecasting-metrics). You could also calculate residuals, like described [here](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-understand-automated-ml#residuals).

The rolling forecast metric values are very high in comparison to the validation metrics reported by the AutoML job. What's going on here? We will investigate in the following cells!

### Forecast versus actuals plot
We will plot predictions and actuals on a time series plot. Since there are many forecasts for each date, we select the 14-day-ahead forecast from each forecast origin for our comparison.

In [ ]:
from matplotlib import pyplot as plt

%matplotlib inline

fcst_df_h14 = (
    fcst_df.groupby("forecast_origin", as_index=False)
    .last()
    .drop(columns=["forecast_origin"])
)
fcst_df_h14.set_index(time_column_name, inplace=True)
plt.plot(fcst_df_h14[[target_column_name, "predicted"]])
plt.xticks(rotation=45)
plt.title(f"Predicted vs. Actuals")
plt.legend(["actual", "14-day-ahead forecast"])
plt.show()

Looking at the plot, there are two clear issues:

1. An anomalously low count value on October 29th, 2012.
2. End-of-year holidays (Thanksgiving and Christmas) in late November and late December.

What happened on Oct. 29th, 2012? That day, Hurricane Sandy brought severe storm surge flooding to the east coast of the United States, particularly around New York City. This is certainly an anomalous event that the model did not account for!

As for the late year holidays, the model apparently did not learn to account for the full reduction of solar panels sold on these major holidays. The training data covers 2011 and early 2012, so the model fit only had access to a single occurrence of these holidays. This makes it challenging to resolve holiday effects; however, a larger AutoML model search may result in a better model that is more holiday-aware.

If we filter the predictions prior to the Thanksgiving holiday and remove the anomalous day of 2012-10-29, the metrics are closer to validation levels:

In [ ]:
date_filter = (fcst_df.date != "2012-10-29") & (fcst_df.date < "2012-11-22")
scores = scoring.score_regression(
    y_test=fcst_df[date_filter][target_column_name],
    y_pred=fcst_df[date_filter]["predicted"],
    metrics=list(constants.Metric.SCALAR_REGRESSION_SET),
)

print("[Test data scores (filtered)]\n")
for key, value in scores.items():
    print("{}:   {:.3f}".format(key, value))